This data was pulled from the UCI Respository and is a REAL dataset
go to http://archive.ics.uci.edu/ml/datasets/HCC+Survival for more information!

Abstract: Hepatocellular Carcinoma dataset (HCC dataset) 
was collected at a University Hospital in Portugal. 
It contains real clinical data of 165 patients diagnosed with HCC. This project will be 
an interesting look into doing classification with pretty small data, only 165 points.
With almost 50 features (dimensions), this is a hard problem!

It's also a good problem on tackling datasets that are small, as doing maximum likelihood
estimation on small data can give results that might not be flexible for future prediction, while a Bayesian inference will give the whole distribution to play with and do some analysis. The Bayesian contet also lets us put some prior information (even if it isn't so strong), which can give us an edge over maximum likelihood when we don't have a lot of data, especially disease data. 


In [1]:
# First, we read in the file and import packages
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import seaborn as sns
import pymc3 as pm
from sklearn.metrics import confusion_matrix as cmatrix


In [2]:
# Load the HCC data
hcc_data = pd.read_csv('/Users/rabeya/Carcinoma_Classification_data/hcc.csv')

In [3]:
hcc_data

,1.Gen,2.Sym,3.Alc,4.HepB,5.HepB,6.HepB,7.HepC,8.Cir,9.End,10.Smo,...,41.Alk,42.Prot,43.Crea,44.NNod,45.dnod,46.Bil,47.Iro,48.Oxy,49.Fer,Class
0,1,0,1,0,0,0,0,1,0,1,...,150,7.1,0.7,1,3.5,0.5,?,?,?,1
1,0,?,0,0,0,0,1,1,?,?,...,?,?,?,1,1.8,?,?,?,?,1
2,1,0,1,1,0,1,0,1,0,1,...,109,7,2.1,5,13,0.1,28,6,16,1
3,1,1,1,0,0,0,0,1,0,1,...,174,8.1,1.11,2,15.7,0.2,?,?,?,0
4,1,1,1,1,0,1,0,1,0,1,...,109,6.9,1.8,1,9,?,59,15,22,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,0,0,1,?,?,?,1,1,0,1,...,109,7.6,0.7,5,3,?,?,?,?,1
161,0,1,0,?,?,?,?,1,0,0,...,280,6.7,0.7,1,2.2,2.3,?,?,?,0
162,1,0,1,0,0,0,0,1,0,1,...,181,7.5,1.46,5,18.6,?,?,?,?,1
163,1,0,1,1,0,1,1,1,1,1,...,170,8.4,0.74,5,18,?,?,?,?,0


According to the UCI website,
this is an heterogeneous dataset, with 23 quantitative variables, 
and 26 qualitative variables. Overall, missing data represents 10.22% 
of the whole dataset and only eight patients have complete information 
in all fields (4.85%). The target variables is the survival at 1 year, 
and was encoded as a binary variable: 0 (dies) and 1 (lives). 
A certain degree of class-imbalance is also present (63 cases labeled as "dies" 
and 102 as "alive").

Our strategy for creating a classification algorithm: since there isnt a lot
of data points (only 165), doesn't make sense to use heavy-duty shit. Maybe a
random forest could work here, but we can always start with the simplest classification
model:: logistic regression. BUT -- were gonna add a bayesian twist to it.

First we'll select enough features we think are basically independent, and 
calculate the covariance matrix of those features. we will then use a prior 
distribution on those features (parameters) -- which will be a Gaussian with a mean
vector with the means of the features, and variance as the covariance matrix. 

Using this prior and the data, we will do bayesian inference on logistic regression
to make predictions for the remainder of the dataset.

# Part 1: Exploratory Data Analysis (EDA) –– Fixing Missing Values

First we need to fix some missing data; either we replace the missing one with the
mean around the others, or drop it, or some other shit. We'll use Miria santos's method of the HEOM distance to do this. 

In [4]:
hcc_data.columns

Index(['1.Gen', '2.Sym', '3.Alc', '4.HepB', '5.HepB', '6.HepB', '7.HepC',
       '8.Cir', '9.End', '10.Smo', '11.Dia', '12.Obe', '13.Hem', '14.Art',
       '15.CRen', '16.HIV', '17.Non', '18.EVar', '19.Spl', '20.PHyp', '21.Thr',
       '22.LMet', '23.Rad', '24.Agedia', '25.Alcpd', '26.cigpy', '27.Sta',
       '28.Encdeg', '29.Ascdeg', ' 30.IntNorRat', ' 31.Alp', ' 32.Hae',
       ' 33.MCorVol', ' 34.Leu', '35.Plat', '36.Alb', '37.Bil', '38.Ala',
       '39.Aspa', '40.Gam', '41.Alk', '42.Prot', '43.Crea', '44.NNod',
       '45.dnod', '46.Bil', '47.Iro', '48.Oxy', '49.Fer', 'Class'],
      dtype='object')

In [5]:
# in this cell, we have a feature data_type dictionary from the UCI repository 
nominal = 'nominal'
ordinal = 'ordinal'
integer = 'integer'
continuous = 'continuous'
ftypes = {0: nominal,
1: nominal,
2: nominal,
3: nominal,
4: nominal,
5: nominal,
6: nominal,
7: nominal,
8: nominal,
9: nominal,
10: nominal,
11: nominal,
12: nominal,
13: nominal,
14: nominal,
15: nominal,
16: nominal,
17: nominal,
18: nominal,
19: nominal,
20: nominal,
21: nominal,
22: nominal,
23: integer,
24: continuous,
25: continuous,
26: ordinal,
27: ordinal,
28: ordinal,
29: continuous,
30: continuous,
31: continuous,
32: continuous,
33: continuous,
34: continuous,
35: continuous,
36: continuous,
37: continuous,
38: continuous,
39: continuous,
40: continuous,
41: continuous,
42: continuous,
43: integer,
44: continuous,
45: continuous,
46: continuous,
47: continuous,
48: continuous,
49: nominal}

In [6]:
# this cell programs the distance functions we use for fixing the missing values
import math
def HEOM_dist(rowA, rowB, df):
    dist_vec = np.asarray(dist_list(rowA, rowB,df))
    return np.linalg.norm(dist_vec)

def dist_list(vecA, vecB, df):
    L = []
    for j in range(0,len(df.columns)-1):
        # if the attribute is missing from vecA or vecB
        if ((vecA[j] == '?') or (vecB[j] == '?')):
            L.append(1)
        else:
            # if the attribute is there and the feature is discrete
            if ((ftypes[j] == 'nominal') or (ftypes[j] == 'ordinal') or (ftypes[j] == 'integer')):
                if (float(vecA[j]) == float(vecB[j])):
                    L.append(0)
                else:
                    L.append(1)
            # if the attribute is there and the feature is continuous
            elif (ftypes[j] == 'continuous'):
                numerator = abs(float(vecA[j])-float(vecB[j]))
                # denominator is tricky. 
                # first make a copy of df.iloc[:,j]  
                copy = df.iloc[:,j].copy().values
                # then delete '?'
                copy = np.delete(copy, np.where(copy=='?'))
                # finally convert to float
                copy = copy.astype(np.float)
                denominator = max(copy)-min(copy)
                ans = numerator/denominator
                L.append(ans)
    # return the list of distances (based on the coordinate, j)
    return L        

In [7]:
# Now let's fix the missing values!
# for each row-vector v, check if there's missing values
# if there is, we take distancees between v and the set of remaining rows
    # we choose the row-vector w closest, and fill in the missing value
# if v has no missing-values at all, we continue in the loop

# initialize the new empty dataframe (0 rows, all columns)
new_df = pd.DataFrame(columns=hcc_data.columns[:-1])

for i in hcc_data.index:
    # each row 
    row_i = hcc_data.iloc[i, :][:-1]
    if ('?' in row_i.values):
        # if the row does have any missing values:
        heom_distances = []
        rest = hcc_data.index.delete(i)
        for j in rest:
            # jth-vector is row_j
            row_j = hcc_data.iloc[j,:][:-1]
            # before calculating HEOM, check row_j has complete info in the wanted features
            # if row_j does, calculate HEOM. if not, skip over it.
            # check where the '?' is in row_i, and if row_j has complete data in the same spots
            row_i_missing = np.where(row_i.values == '?')[0]
            if ('?' not in row_j.values[row_i_missing]):
                # if row_j has no missing values in the same spots,
                # then compute the HEOM distance
                D = HEOM_dist(row_i, row_j, hcc_data)
                heom_distances.append((j,D))
            else:
                # otherwise skip this vector and continue with the loop
                continue
                
        # after the distances are computed and stored in a list:       
        # closest vector to row_i according to distance D
        closest_ind = min(heom_distances, key=lambda x: x[1])[0]
        # we need to fill in the missing attributes in row_i from closest_v
        closest_v = hcc_data.iloc[closest_ind, :][:-1].values # this is the row-vector in hcc_data
        # replace the missing attributes from row_i with the corresponding ones from
        # closest_v
        # first we get the indices of the missing_value (the '?' value)
        missing_indexes = np.where(row_i.values == '?')[0]
        # then we get the corresponding values from closest_v
        corresponding_vals = closest_v[missing_indexes.tolist()]
        # finally we replace the values in row_i with the new corresponding values
        row_i[missing_indexes] = corresponding_vals
        # finally update the empty dataframe by adding the row
        new_df = new_df.append(row_i, ignore_index=True)
    else:
        # if there is no missing attribute values in row_i
        # just update the new_df dataframe by adding the row
        new_df = new_df.append(row_i, ignore_index=True)
            
   

In [8]:
new_df

,1.Gen,2.Sym,3.Alc,4.HepB,5.HepB,6.HepB,7.HepC,8.Cir,9.End,10.Smo,...,40.Gam,41.Alk,42.Prot,43.Crea,44.NNod,45.dnod,46.Bil,47.Iro,48.Oxy,49.Fer
0,1,0,1,0,0,0,0,1,0,1,...,183,150,7.1,0.7,1,3.5,0.5,52.5,37,856
1,0,0,0,0,0,0,1,1,0,0,...,663,433,6.5,0.87,1,1.8,0.4,52.5,37,856
2,1,0,1,1,0,1,0,1,0,1,...,202,109,7,2.1,5,13,0.1,28,6,16
3,1,1,1,0,0,0,0,1,0,1,...,94,174,8.1,1.11,2,15.7,0.2,52.5,37,856
4,1,1,1,1,0,1,0,1,0,1,...,173,109,6.9,1.8,1,9,0.1,59,15,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,0,0,1,0,0,1,1,1,0,1,...,449,109,7.6,0.7,5,3,0.2,87,26,84
161,0,1,0,0,0,1,0,1,0,0,...,147,280,6.7,0.7,1,2.2,2.3,94,37,48
162,1,0,1,0,0,0,0,1,0,1,...,164,181,7.5,1.46,5,18.6,0.85,32,10,18
163,1,0,1,1,0,1,1,1,1,1,...,320,170,8.4,0.74,5,18,1.8,161,96,297


In [9]:
# yay! no missing data anymore in the new_df dataframe.
# let's start balancing the data.

# Part 2: EDA –– Class Balance

The data we have is a bit unbalanced, since there's only around 63 samples of death and 102 cases of life. So we should try to balance the dataset (ideally by a combination of over and undersampling) so the dataset is balanced. At the same time, we shouldn't feel the need to generate (from SMOTE) so many artifical cases, because the more cases, the algorithm will "overfit" on the excessive over-sampled artifical examples (credit to @horaceT on Stats.stackexchange for that explanation). The risks to undersampling is losing data.

We don't necessarily have to be too worried about sampling too much since we only have 165 data points. However, we don't want to create too much artifical data since we want to algorithm to classify and not overfit too much. Also, it's why we are using a Bayesian approach, since Bayesian analysis can be more useful on small data if we can use priors with some information.

In [10]:
new_df['Class'] = hcc_data['Class']
new_df

,1.Gen,2.Sym,3.Alc,4.HepB,5.HepB,6.HepB,7.HepC,8.Cir,9.End,10.Smo,...,41.Alk,42.Prot,43.Crea,44.NNod,45.dnod,46.Bil,47.Iro,48.Oxy,49.Fer,Class
0,1,0,1,0,0,0,0,1,0,1,...,150,7.1,0.7,1,3.5,0.5,52.5,37,856,1
1,0,0,0,0,0,0,1,1,0,0,...,433,6.5,0.87,1,1.8,0.4,52.5,37,856,1
2,1,0,1,1,0,1,0,1,0,1,...,109,7,2.1,5,13,0.1,28,6,16,1
3,1,1,1,0,0,0,0,1,0,1,...,174,8.1,1.11,2,15.7,0.2,52.5,37,856,0
4,1,1,1,1,0,1,0,1,0,1,...,109,6.9,1.8,1,9,0.1,59,15,22,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,0,0,1,0,0,1,1,1,0,1,...,109,7.6,0.7,5,3,0.2,87,26,84,1
161,0,1,0,0,0,1,0,1,0,0,...,280,6.7,0.7,1,2.2,2.3,94,37,48,0
162,1,0,1,0,0,0,0,1,0,1,...,181,7.5,1.46,5,18.6,0.85,32,10,18,1
163,1,0,1,1,0,1,1,1,1,1,...,170,8.4,0.74,5,18,1.8,161,96,297,0


In [11]:
# next, before we do the matrix, we need to convert everything into float
for col in new_df.columns:
    new_df[col] = pd.to_numeric(new_df[col], errors='coerce')
new_df.head()

,1.Gen,2.Sym,3.Alc,4.HepB,5.HepB,6.HepB,7.HepC,8.Cir,9.End,10.Smo,...,41.Alk,42.Prot,43.Crea,44.NNod,45.dnod,46.Bil,47.Iro,48.Oxy,49.Fer,Class
0,1,0,1,0,0,0,0,1,0,1,...,150.0,7.1,0.70,1,3.5,0.5,52.5,37.0,856.0,1
1,0,0,0,0,0,0,1,1,0,0,...,433.0,6.5,0.87,1,1.8,0.4,52.5,37.0,856.0,1
2,1,0,1,1,0,1,0,1,0,1,...,109.0,7.0,2.10,5,13.0,0.1,28.0,6.0,16.0,1
3,1,1,1,0,0,0,0,1,0,1,...,174.0,8.1,1.11,2,15.7,0.2,52.5,37.0,856.0,0
4,1,1,1,1,0,1,0,1,0,1,...,109.0,6.9,1.80,1,9.0,0.1,59.0,15.0,22.0,1


In [12]:
new_df.shape

(165, 50)

In [13]:
#initialize the samplers
from imblearn.over_sampling import SMOTE
sme = SMOTE(random_state=10)

#features
features = new_df.columns
# transform the dataset
X_matrix = new_df[features[:-1]].values
y_matrix = new_df[features[-1]].values

Using TensorFlow backend.
/Users/rabeya/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/rabeya/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/rabeya/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/rabeya/opt/anaconda3/lib/python3.7

In [14]:
X_matrix.shape, y_matrix.shape

((165, 49), (165,))

In [15]:
# time to do the sampling!
X_trans, y_trans = sme.fit_resample(X_matrix, y_matrix)

In [16]:
from collections import Counter

Counter(y_trans)

Counter({1: 102, 0: 102})

In [17]:
transformed_df = pd.DataFrame(X_trans, columns=features[:-1])
transformed_df['Class'] = new_df['Class']

In [18]:
transformed_df.shape

(204, 50)

In [19]:
transformed_df.head()

,1.Gen,2.Sym,3.Alc,4.HepB,5.HepB,6.HepB,7.HepC,8.Cir,9.End,10.Smo,...,41.Alk,42.Prot,43.Crea,44.NNod,45.dnod,46.Bil,47.Iro,48.Oxy,49.Fer,Class
0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,150.0,7.1,0.70,1.0,3.5,0.5,52.5,37.0,856.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,433.0,6.5,0.87,1.0,1.8,0.4,52.5,37.0,856.0,1.0
2,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,109.0,7.0,2.10,5.0,13.0,0.1,28.0,6.0,16.0,1.0
3,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,174.0,8.1,1.11,2.0,15.7,0.2,52.5,37.0,856.0,0.0
4,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,109.0,6.9,1.80,1.0,9.0,0.1,59.0,15.0,22.0,1.0


# Part 4: Bayesian Inference with PYMC3

Let's do Bayesian logistic regression!
We will follow a paper in 2008 (Andrew Gelman) for guidance on the prior distirbution. Since we don't have a lot of data, but we have many features (almost 50 features!) -- we will use Theano tensors and isolate the relevant variables.

This might be a bit confusing, but the logistic-regression equation describes the probability of the class being 1 (Alive) or 0 (Death) GIVEN  the data we have. So actually, in the Bayesian framework, the logitic function/regression is our likelihood function because it's equal to pr(Y=1 or 0|Data).

In [20]:
y_trans

array([1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

In [21]:
# let's use scikit SelectK Best features (using mutual information) to determine "the best"
# 12 "best" parameters/features to select from the 49 parameters we have
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
K = 12
selector = SelectKBest(mutual_info_classif, k=K)
X_reduced = selector.fit_transform(X_trans, y_trans)
features_selected = selector.get_support()

In [22]:
X_reduced.shape, y_trans.shape

((204, 12), (204,))

In [23]:
import theano
import theano.tensor as tt
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X_reduced, y_trans, train_size=0.4)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((81, 12), (81,), (123, 12), (123,))

In [24]:
# lets put the training and test data wrapped in a theano tensor
# for computation later
model_input = theano.shared(X_train)
model_output = theano.shared(y_train)

In [25]:
X_train.shape, y_train.shape

((81, 12), (81,))

In [26]:
def tinvlogit(x):
    return tt.exp(x) / (1 + tt.exp(x))

In [48]:
# First we will create our prior distributions over the K parameters 
# and then use the logistic regression equation as our likelihood.

# create the model
with pm.Model() as logistic_model:
    # laplace prior
    betas = pm.Laplace('betas', mu=0, b=5)
    
    # probability prameter
    prob = tinvlogit(constant + tt.sum(betas*model_input, axis=1)) 
    #likelihood function below 
    likelihood = pm.Bernoulli('like', p=prob, observed=model_output)

In [52]:
# this is MCMC approximation
with logistic_model:
    #time for inference!
    trace = pm.sample(draws=6000, init='adapt_diag' tune=12000)


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


MissingInputError: Input 0 of the graph (indices start from 0), used to compute Elemwise{add,no_inplace}(constant, Sum{axis=[1], acc_dtype=float64}.0), was not provided and not given a value. Use the Theano flag exception_verbosity='high', for more information on this error.

In [ ]:
# Create posterior predictive (PPC) samples from ADVI
ppc = pm.sample_ppc(advi_trace, model=logistic, samples=1000)

#plotting the posterior predictive check samples (PPC)
_, ax = plt.subplots(figsize=(12, 6))
ax.hist([n.mean() for n in ppc['n']], bins=19, alpha=0.5)
ax.axvline(data.mean())
ax.set(title='Posterior predictive of the mean', xlabel='mean(x)', ylabel='Frequency');

In [ ]:
pm.summary(trace)
#plt.show()